In [1]:
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt
import os

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import threading
from ultralytics import YOLO
# ===== Detection summary box =====
from tkinter import scrolledtext
from collections import Counter
import torch
import pyttsx3

# Initialize TTS engine
tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 150)  # Optional: set speaking rate

# Global dictionary to store latest detection result
latest_detection_summary = {
    "notes": {},       # e.g., {"RM10": 2, "RM1": 1}
    "total_amount": 0  # e.g., 21
}

# Define class name to monetary value mapping
currency_values = {
    "RM1": 1,
    "RM5": 5,
    "RM10": 10,
    "RM20": 20,
    "RM50": 50,
    "RM100": 100
}

# Load YOLOv8 model
model = YOLO("colab/train_results/weights/best.pt")  # Update this path as needed

def update_detection_summary(results):
    global latest_detection_summary
    try:
        cls_tensor = results[0].boxes.cls
        cls_list = cls_tensor.tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor
        counts = Counter(cls_list)

        # Summary string and total computation
        summary = ""
        total_amount = 0
        detected_notes = {}

        for class_id, count in counts.items():
            class_name = model.names[int(class_id)]
            note_value = currency_values.get(class_name, 0)
            detected_notes[class_name] = count
            subtotal = note_value * count
            total_amount += subtotal
            summary += f"{class_name}: {count} x RM{note_value} = RM{subtotal}\n"

        summary += f"\nTotal Amount: RM{total_amount}"

        # Update GUI text box
        detection_box.config(state=tk.NORMAL)
        detection_box.delete(1.0, tk.END)
        detection_box.insert(tk.END, summary.strip())
        detection_box.config(state=tk.DISABLED)

        # Store in global dictionary
        latest_detection_summary["notes"] = detected_notes
        latest_detection_summary["total_amount"] = total_amount

    except Exception as e:
        detection_box.config(state=tk.NORMAL)
        detection_box.delete(1.0, tk.END)
        detection_box.insert(tk.END, f"Error: {str(e)}")
        detection_box.config(state=tk.DISABLED)

def speak_detection_summary():
    notes = latest_detection_summary.get("notes", {})
    total = latest_detection_summary.get("total_amount", 0)

    if not notes:
        tts_engine.say("No currency notes detected.")
    else:
        speech_output = "Detected currency notes: "
        for note, count in notes.items():
            speech_output += f"{count} piece{'s' if count > 1 else ''} of {note}, "
        speech_output += f"Total amount is {total} ringgit."
        tts_engine.say(speech_output)

    tts_engine.runAndWait()


# GUI setup
root = tk.Tk()
root.title("YOLOv8 GUI Demo")
root.geometry("1024x768")

# State variables
cap = None
running = False

# Helper: Show image in GUI
def show_image(img_cv2):
    img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    img_tk = ImageTk.PhotoImage(img_pil)
    panel.imgtk = img_tk
    panel.configure(image=img_tk)

# Prediction: Run YOLO on static image
def predict_image(path):
    results = model(path)
    img = results[0].plot()
    show_image(img)
    update_detection_summary(results)

# Prediction: Open image
def upload_image():
    path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.jpeg")])
    if path:
        predict_image(path)

# Webcam handling
def webcam_loop():
    global cap, running
    while running:
        ret, frame = cap.read()
        if not ret:
            break

        # 👇 Copy and flip the frame horizontally (mirror)
        frame_flipped = cv2.flip(frame.copy(), 1)  # 1 = horizontal flip

        # Run YOLOv8 on the flipped frame
        results = model(frame_flipped)

        # Draw bounding boxes
        img = results[0].plot()

        # Display
        show_image(img)
        update_detection_summary(results)

        if cv2.waitKey(1) == ord('q'):
            break



# Start webcam
def start_webcam():
    global cap, running
    stop_all_streams()  # Stop other sources
    cap = cv2.VideoCapture(0)
    running = True
    threading.Thread(target=webcam_loop, daemon=True).start()

# Video playback loop
def video_loop(path):
    global cap, running
    cap = cv2.VideoCapture(path)
    running = True
    while running and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame)
        img = results[0].plot()
        show_image(img)
        update_detection_summary(results)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break
    cap.release()

# Upload and play video
def upload_video():
    path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi *.mov")])
    if path:
        stop_all_streams()
        threading.Thread(target=video_loop, args=(path,), daemon=True).start()

# Stop all video/camera streams
def stop_all_streams():
    global cap, running
    running = False
    if cap:
        cap.release()
        cap = None

# Buttons
btn_frame = tk.Frame(root)
btn_frame.pack(pady=10)

# ===== Button Panel on the Left =====
button_panel = tk.Frame(root, width=200, bg='lightgray')
button_panel.pack(side="left", fill="y")

btn_upload_img = tk.Button(button_panel, text="Upload Image", command=upload_image)
btn_upload_img.pack(pady=10, padx=10, fill="x")

btn_upload_vid = tk.Button(button_panel, text="Upload Video", command=upload_video)
btn_upload_vid.pack(pady=10, padx=10, fill="x")

btn_cam_start = tk.Button(button_panel, text="Start Webcam", command=start_webcam)
btn_cam_start.pack(pady=10, padx=10, fill="x")

btn_cam_stop = tk.Button(button_panel, text="Stop Webcam", command=stop_all_streams)
btn_cam_stop.pack(pady=10, padx=10, fill="x")

btn_speak_result = tk.Button(button_panel, text="Speak Result", command=speak_detection_summary)
btn_speak_result.pack(pady=10, padx=10, fill="x")


detection_box = scrolledtext.ScrolledText(button_panel, height=15, width=25, wrap=tk.WORD)
detection_box.pack(pady=10, padx=10, fill="both")
detection_box.insert(tk.END, "Detection summary will appear here.")
detection_box.config(state=tk.DISABLED)

# ===== Display Panel on the Right =====
display_panel = tk.Frame(root, bg='black')
display_panel.pack(side="right", expand=True, fill="both")

panel = tk.Label(display_panel, bg='black')
panel.pack(expand=True)

# Run GUI
root.mainloop()



image 1/1 D:\Projects-Ng\Currency_recognition\assets\image1.jpg: 320x640 1 RM100, 1596.1ms
Speed: 376.8ms preprocess, 1596.1ms inference, 221.6ms postprocess per image at shape (1, 3, 320, 640)

0: 480x640 (no detections), 298.2ms
Speed: 74.3ms preprocess, 298.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 188.1ms
Speed: 3.2ms preprocess, 188.1ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 RM1, 220.4ms
Speed: 31.9ms preprocess, 220.4ms inference, 41.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 275.5ms
Speed: 2.8ms preprocess, 275.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 256.9ms
Speed: 5.4ms preprocess, 256.9ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 264.8ms
Speed: 3.2ms preprocess, 264.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0

0: 480x640 (no detections), 1161.8ms
Speed: 2.7ms preprocess, 1161.8ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)


Exception in thread Thread-18 (webcam_loop):
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "d:\Projects-Ng\Currency_recognition\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\User\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_19800\1038272056.py", line 138, in webcam_loop
    show_image(img)
    ~~~~~~~~~~^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_19800\1038272056.py", line 103, in show_image
    img_tk = ImageTk.PhotoImage(img_pil)
  File "d:\Projects-Ng\Currency_recognition\.venv\Lib\site-packages\PIL\ImageTk.py", line 129, in __init__
    self.__photo = tkint